# PydanticOutputParser

`PydanticOutputParser`는 언어 모델의 출력을 **구조화된 정보**로 변환하는 데 도움을 주는 클래스임. 

이 클래스는 단순 텍스트 응답 대신 **명확하고 체계적인 형태로 필요한 정보를 제공**할 수 있음.

이 클래스를 활용하면 언어 모델의 출력을 특정 데이터 모델에 맞게 변환하여 정보를 더 쉽게 처리하고 활용할 수 있음.

## 주요 메서드

`PydanticOutputParser` (대부분의 OutputParser에 해당)에는 주로 **두 가지 핵심 메서드**가 구현되어야 합니다.

- **`get_format_instructions()`**: 언어 모델이 출력해야 할 정보의 형식을 정의하는 지침을 제공합니다. 예를 들어, 언어 모델이 출력해야 할 데이터의 필드와 그 형태를 설명하는 지침을 문자열로 반환할 수 있습니다. 이 지침은 언어 모델이 출력을 구조화하고 특정 데이터 모델에 맞게 변환하는 데 매우 중요함.
- **`parse()`**: 언어 모델의 출력(문자열로 가정)을 받아 이를 특정 구조로 분석하고 변환합니다. Pydantic과 같은 도구를 사용하여 입력된 문자열을 사전 정의된 스키마에 따라 검증하고, 해당 스키마를 따르는 데이터 구조로 변환함.

## 참고 자료

- [Pydantic 공식 도큐먼트](https://docs.pydantic.dev/latest/)


In [64]:
from dotenv import load_dotenv

load_dotenv()

True

In [65]:
# !pip install langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("01-PydanticOuputParser")

LangSmith 추적을 시작합니다.
[프로젝트명]
01-PydanticOuputParser


In [66]:
# 실시간 출력을 위한 import
from langchain_teddynote.messages import stream_response

In [68]:
from langchain_openai import AzureChatOpenAI
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field

다음은 이메일 본문 예시입니다.


In [34]:
email_conversation = """From: Byung Jo Bang <bjbang@microsoft.com>
To: 신 하섬 <haseom.shinhs@i-esg.io>
Subject: Increased usage of Azure AI Studio LLM models - TrackingID#2411220030003284

안녕하세요.
 
현재 gpt-4o-mini 모델을 사용하고 있고, limit(2M/분) 을 넘어서 429 오류(Too many requests)가 발생하고 있는 것으로 이해가 됩니다.
 
관련하여 몇 가지 문의 사항 및 요청 사항이 있습니다.
1. 혹시 오류가 발생하고 있는 Azure OpenAI 리소스의 이름과 사용하는 리전은 어떻게 되나요?
 
2. 추가로, 429 error 가 발생한다고 하였는데, 쿼터 신청 페이지에서 쿼터 증설 요청을 하여 보실 수 있습니다.
아래는 쿼터 증설 요청을 할 수 있는 사이트인데, 증설 요청을 해 보실 수 있을까요?
Azure OpenAI Service: Request for Quota Increase
 
3. 현재 이 케이스를 생성하실 때에 24x7 지원으로 케이스가 생성 되어 있습니다.
24x7 버튼이 enable되어 있는 경우에는 한국 시간으로 오후 5시가 넘는 경우에 다른 time zone으로 전달이 되며, 야간과 주간 시간 동안, 즉 24시간동안 유럽, 미국, 아시아에 있는 영어 지원만 가능한 엔지니어에 의해서 계속 지원이 됩니다. 이 때에 고객에게도 연락이 갈 수 있어서 야간 시간 동안에 메일이나 전화에 응답을 해 주셔야 합니다. 그렇지만, 현 케이스의 경우에는 따로 전화 번호가 제공이 되지 않고 있어서 24x7 지원은disable을 해 놓아야 할 것 같습니다. 이 점에 대해서는 널리 이해를 부탁드립니다.
혹시 24x7 지원이 필요하다면, 따로 전화 번호를 제공하여 주실 수 있을까요? 그렇다면, 다른 리전으로 이 케이스를 전달해 드리도록 하겠습니다.
만약 24x7 지원이 필요 없으시다면 말씀을 부탁드리겠습니다.
 
위의 사항에 대해서 혹시 궁금한 사항이 있으면 메일을 주시거나 또는 전화 번호를 알려 주시면, 따로 연락을 드리도록 하겠습니다.
 
 
좋은 하루 되세요.
감사합니다.
"""

출력 파서를 사용하지 않는 경우 예시

In [69]:
from itertools import chain
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    "다음의 이메일 내용중 중요한 내용을 추출해 주세요.\n\n{email_conversation}"
)

llm = AzureChatOpenAI(temperature=0, model_name="gpt-4o")

chain = prompt | llm

answer = chain.stream({"email_conversation": email_conversation})

output = stream_response(answer, return_output=True)

이메일의 중요한 내용은 다음과 같습니다:

1. **모델 사용 현황**: 현재 사용 중인 모델은 gpt-4o-mini이며, 요청 한도를 초과하여 429 오류(Too many requests)가 발생하고 있음.

2. **문의 사항**:
   - Azure OpenAI 리소스의 이름과 사용하는 리전 정보 요청.

3. **쿼터 증설 요청**: 429 오류 해결을 위해 Azure OpenAI Service의 쿼터 증설 요청 페이지를 통해 증설 요청을 할 수 있음.

4. **24x7 지원 관련**:
   - 현재 케이스는 24x7 지원으로 생성되어 있으며, 한국 시간으로 오후 5시 이후에는 다른 시간대의 지원 엔지니어가 대응.
   - 고객에게 연락이 갈 수 있으므로, 야간 시간 동안 메일이나 전화에 응답 필요.
   - 전화 번호가 제공되지 않으면 24x7 지원을 비활성화해야 함.
   - 24x7 지원이 필요할 경우 전화 번호 제공 요청.

5. **연락처**: 추가 질문이 있을 경우 이메일 또는 전화 번호를 통해 연락 가능.

In [36]:
print(output)


이메일의 중요한 내용은 다음과 같습니다:

1. **모델 사용 현황**: 현재 사용 중인 모델은 gpt-4o-mini이며, 요청 한도를 초과하여 429 오류(Too many requests)가 발생하고 있음.

2. **문의 사항**:
   - Azure OpenAI 리소스의 이름과 사용하는 리전 정보 요청.

3. **쿼터 증설 요청**: 429 오류 해결을 위해 Azure OpenAI Service의 쿼터 증설 요청 페이지를 통해 증설 요청을 할 수 있음.

4. **24x7 지원 관련**:
   - 현재 케이스는 24x7 지원으로 생성되어 있으며, 한국 시간으로 오후 5시 이후에는 다른 시간대의 지원 엔지니어가 대응.
   - 고객에게 연락이 갈 수 있으므로, 야간 시간 동안 메일이나 전화에 응답 필요.
   - 전화 번호가 제공되지 않으면 24x7 지원을 비활성화해야 함.
   - 24x7 지원이 필요할 경우 전화 번호 제공 요청.

5. **연락처**: 추가 질문이 있을 경우 이메일 또는 전화 번호를 통해 연락 가능.


위와 같은 이메일 내용이 주어졌을 때 아래의 Pydantic 스타일로 정의된 클래스를 사용하여 이메일의 정보를 파싱.

참고로, Field 안에 `description` 은 텍스트 형태의 답변에서 주요 정보를 추출하기 위한 설명임. LLM 이 바로 이 설명을 보고 필요한 정보를 추출하게 됨. 그러므로 이 설명은 정확하고 명확해야 함.

In [70]:
class EmailSummary(BaseModel):
    person: str = Field(description="메일을 보낸 사람")
    email: str = Field(description="메일을 보낸 사람의 이메일 주소")
    subject: str = Field(description="메일 제목")
    summary: str = Field(description="메일 본문을 요약한 텍스트")
    requests: str = Field(description="메일 본문에서 요청하는 상세한 요청 사항")


# PydanticOutputParser 생성
parser = PydanticOutputParser(pydantic_object=EmailSummary)

In [71]:
# instruction 을 출력합니다.
print(parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"person": {"description": "메일을 보낸 사람", "title": "Person", "type": "string"}, "email": {"description": "메일을 보낸 사람의 이메일 주소", "title": "Email", "type": "string"}, "subject": {"description": "메일 제목", "title": "Subject", "type": "string"}, "summary": {"description": "메일 본문을 요약한 텍스트", "title": "Summary", "type": "string"}, "requests": {"description": "메일 본문에서 요청하는 상세한 요청 사항", "title": "Requests", "type": "string"}}, "required": ["person", "email", "subject", "summary", "requests"]}
```


프롬프트를 정의합니다.

1. `question`: 유저의 질문을 받습니다.
2. `email_conversation`: 이메일 본문의 내용을 입력합니다.
3. `format`: 형식을 지정합니다.


In [72]:
prompt = PromptTemplate.from_template(
    """
You are a helpful assistant. Please answer the following questions in KOREAN.

QUESTION:
{question}

EMAIL CONVERSATION:
{email_conversation}

FORMAT:
{format}
"""
)

# format 에 PydanticOutputParser의 부분 포맷팅(partial) 추가
prompt = prompt.partial(format=parser.get_format_instructions())

다음으로는 Chain 을 생성합니다.


In [73]:
# chain 을 생성합니다.
chain = prompt | llm

체인을 실행하고 결과를 확인합니다.


In [74]:
# chain 을 실행하고 결과를 출력합니다.
response = chain.stream(
    {
        "email_conversation": email_conversation,
        "question": "이메일 내용중 주요 내용을 추출해 주세요.",
    }
)

# 결과는 JSON 형태로 출력됩니다.
output = stream_response(response, return_output=True)

```json
{
  "person": "Byung Jo Bang",
  "email": "bjbang@microsoft.com",
  "subject": "Increased usage of Azure AI Studio LLM models - TrackingID#2411220030003284",
  "summary": "gpt-4o-mini 모델 사용 중 429 오류가 발생하고 있으며, 관련하여 Azure OpenAI 리소스 정보와 쿼터 증설 요청, 24x7 지원 여부에 대한 문의가 포함되어 있습니다.",
  "requests": "1. Azure OpenAI 리소스의 이름과 사용하는 리전 정보 요청. 2. 쿼터 증설 요청 사이트 링크 제공 및 요청 가능 여부 확인. 3. 24x7 지원 필요 여부 및 전화 번호 제공 요청."
}
```

마지막으로 `parser`를 사용하여 결과를 파싱하고 `EmailSummary` 객체로 변환합니다.


In [75]:
# PydanticOutputParser 를 사용하여 결과를 파싱합니다.
structured_output = parser.parse(output)
print(structured_output)

person='Byung Jo Bang' email='bjbang@microsoft.com' subject='Increased usage of Azure AI Studio LLM models - TrackingID#2411220030003284' summary='gpt-4o-mini 모델 사용 중 429 오류가 발생하고 있으며, 관련하여 Azure OpenAI 리소스 정보와 쿼터 증설 요청, 24x7 지원 여부에 대한 문의가 포함되어 있습니다.' requests='1. Azure OpenAI 리소스의 이름과 사용하는 리전 정보 요청. 2. 쿼터 증설 요청 사이트 링크 제공 및 요청 가능 여부 확인. 3. 24x7 지원 필요 여부 및 전화 번호 제공 요청.'


## parser 가 추가된 체인 생성

출력 결과를 정의한 Pydantic 객체로 생성할 수 있습니다.

In [76]:
# 출력 파서를 추가하여 전체 체인을 재구성합니다.
chain = prompt | llm | parser

In [77]:
# chain 을 실행하고 결과를 출력합니다.
response = chain.invoke(
    {
        "email_conversation": email_conversation,
        "question": "이메일 내용중 주요 내용을 추출해 주세요.",
    }
)

# 결과는 EmailSummary 객체 형태로 출력됩니다.
response

EmailSummary(person='Byung Jo Bang', email='bjbang@microsoft.com', subject='Increased usage of Azure AI Studio LLM models - TrackingID#2411220030003284', summary='gpt-4o-mini 모델 사용 중 429 오류가 발생하고 있으며, 관련하여 Azure OpenAI 리소스 정보와 쿼터 증설 요청, 24x7 지원 여부에 대한 문의가 포함되어 있습니다.', requests='1. Azure OpenAI 리소스의 이름과 사용하는 리전 정보 요청. 2. 쿼터 증설 요청 사이트 링크 제공 및 요청 가능 여부 확인. 3. 24x7 지원 필요 여부 및 전화 번호 제공 요청.')

## with_structured_output()

`.with_structured_output(Pydantic)`을 사용하여 출력 파서를 추가하면, 출력을 Pydantic 객체로 변환할 수 있습니다.

In [78]:
llm_with_structered = AzureChatOpenAI(
    temperature=0, model_name="gpt-4o"
).with_structured_output(EmailSummary)

In [79]:
# invoke() 함수를 호출하여 결과를 출력합니다.
answer = llm_with_structered.invoke(email_conversation)
answer

EmailSummary(person='Byung Jo Bang', email='bjbang@microsoft.com', subject='Increased usage of Azure AI Studio LLM models - TrackingID#2411220030003284', summary='Byung Jo Bang is addressing issues related to the usage of the gpt-4o-mini model, specifically encountering 429 errors due to exceeding the limit. He has several inquiries and requests regarding the Azure OpenAI resources and support.', requests='1. Request for the name and region of the Azure OpenAI resource causing the 429 errors. 2. A request to submit a quota increase request on the Azure OpenAI Service page. 3. Clarification on the 24x7 support status and the need for a contact number for support.')

In [80]:
print(answer.summary)

Byung Jo Bang is addressing issues related to the usage of the gpt-4o-mini model, specifically encountering 429 errors due to exceeding the limit. He has several inquiries and requests regarding the Azure OpenAI resources and support.


**참고**

한 가지 아쉬운 점은 `.with_structured_output()` 함수는 `stream()` 기능을 지원하지 않습니다.